In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import json
from scipy import ndimage


from readImages import *
from imageModifiers import *
from readDataFiles import *
from projEdgeBreaks import *
from gaussBreaks import *
from saveImages import *
from plottingFuncs import *
from gap_metric_breaks import *

In [2]:
mpl.rcParams["figure.figsize"] = (15, 15)
# %matplotlib inline

subjFile = "data/anti-slavery-manuscripts-subjects.csv"
subj = pd.read_csv(subjFile)

# get metadata in dictionary format
subj["meta_json"] = [json.loads(q) for q in subj["metadata"]]

# get image url
subj["url"] = [json.loads(q).get("0") for q in subj["locations"]]

# get beta images
subj_beta = subj.loc[subj["subject_set_id"] == 16451]
subj_beta = subj_beta.drop_duplicates(["subject_id", "locations"])
len(subj_beta)

11

In [3]:
for i in range(len(subj_beta)):
    # read in and do all pre-processing #################################
    let_orig, grey_orig = readImg(subj_beta["url"].iloc[i])
    grey, let, offset = removeEdges(grey_orig, let_orig, 10.0)
    
    # get linebreaks ####################################################
    greySm = smoothImg(grey, 7.0)
    matchlim = 30
    lb = projBreaks(greySm, "y")
    lb = filterBreaks(lb, matchlim)
    plotBoxes(let, lb, saveFile="example_img/linebreaks"+str(i)+".png")